In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pydataset import data
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression

In [9]:
df = data('swiss')
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [ ]:
# looking at range of values in our dataset ot see if we need to scale

In [10]:
df.describe()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
count,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000
mean,70.142553,50.659574,16.489362,10.978723,41.14383,19.942553
std,12.491697,22.711218,7.977883,9.615407,41.70485,2.912697
min,35.000000,1.200000,3.000000,1.000000,2.15000,10.800000
25%,64.700000,35.900000,12.000000,6.000000,5.19500,18.150000
50%,70.400000,54.100000,16.000000,8.000000,15.14000,20.000000
75%,78.450000,67.650000,22.000000,12.000000,93.12500,21.700000
max,92.500000,89.700000,37.000000,53.000000,100.00000,26.600000


In [ ]:
# fields have different scales. 
# it is wise to scale before continuing!

In [13]:
mms = MinMaxScaler()
to_scale = df.drop(columns=['Fertility']).columns

df[to_scale] = mms.fit_transform(df[to_scale])
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,0.178531,0.352941,0.211538,0.079816,0.721519
Delemont,83.1,0.496045,0.088235,0.153846,0.845069,0.721519
Franches-Mnt,92.5,0.435028,0.058824,0.076923,0.932550,0.594937
Moutier,85.8,0.398870,0.264706,0.115385,0.323148,0.601266
Neuveville,76.9,0.477966,0.411765,0.269231,0.030761,0.620253


lets break our data into x and y subsets.
we will move forward with SelectKBest. This technique uses a statistical test to determine how useful features may be.

In [14]:
X = df.drop(columns = ['Fertility'])
y = df.Fertility

In [15]:
skb = SelectKBest(f_regression, k=3)
skb.fit(X,y)

SelectKBest(k=3, score_func=<function f_regression at 0x7f97f98ac8b0>)

In [18]:
skb_mask = skb.get_support()
X.columns[skb_mask]

Index(['Examination', 'Education', 'Catholic'], dtype='object')

According to SelectKBest: Examination, Education, and Catholic have the dtrongest relationships with my target variable.

In [ ]:
# Now we are workign on RFE
# With RFE we need a model obkect that we will be used to evaluate the predictive pwer of our features.
# We'll use a simple linear regression model ot do this.

In [22]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select=3)

rfe.fit(X,y)

RFE(estimator=LinearRegression(), n_features_to_select=3)

In [24]:
rfe.get_support()

array([ True, False,  True, False,  True])

In [25]:
rfe_mask = rfe.get_support()
X.columns[rfe_mask]

Index(['Agriculture', 'Education', 'Infant.Mortality'], dtype='object')

Our RFE object says that Agriculture, Education, and Infant.Mortality are the top performing features from our dataset.

How does this compare to the features selected by SelectKBest?

SelectKBest and RFE returned diffrent results. This is Ok as they have different evaluation methods.

In [ ]:
If you have a large dataset use RFE rather than SelectKBest. SelectKBest will be computationaly expensive.